In [ ]:
import numpy as np
import pandas as pd
import sklearn.naive_bayes as NB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from google.colab import drive

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tensorflow import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt

drive.mount('/content/gdrive')

In [ ]:
train_ds = pd.read_csv('/content/gdrive/MyDrive/data/std_ANOVA_x_train.csv')
test_ds = pd.read_csv('/content/gdrive/MyDrive/data/std_ANOVA_x_test.csv')
columns = list(train_ds.columns)
# columns.remove('最高學歷')
# columns.remove('畢業學校類別')
train_ds = train_ds.loc[:,columns]
test_ds = test_ds.loc[:,columns]
# test_ds.drop(columns='PerStatus', inplace=True)
# print(len(train_ds))
# print(len(test_ds))

In [ ]:
# nb_train_samples = 5217
# nb_validation_samples = 17
EPOCHS = 100
BATCH_SIZE = 16
INPUT_DIM = 10

In [ ]:
# train_ds.dropna(inplace=True)
test_ds_valid = test_ds.dropna().copy()
test_ds_invalid = test_ds[test_ds.isna().any(axis=1)].copy()


In [ ]:
def F_beta_score(y_true, y_pred, beta=1.5):
    """F beta score with beta=1.5"""
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f_beta = (1+beta**2)*(prec*rec)/(beta**2*prec + rec)
    return f_beta

In [ ]:
#資料準備
# X = train_ds.drop(columns='PerStatus', inplace=False)
X = train_ds
Y = pd.read_csv('/content/gdrive/MyDrive/data/y_train.csv')
Y.dropna(inplace=True)
Y.PerStatus.value_counts() # label distribution

train_x, valid_x, train_y, valid_y = train_test_split(X,Y,test_size = 0.2,random_state=1)

In [ ]:
#不平衡資料計算
number_pos = Y.PerStatus.value_counts()[1]
number_neg = Y.PerStatus.value_counts()[0]
total = number_pos + number_neg
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, number_pos, 100 * number_pos / total))

In [ ]:
#不均勻數據權重分配 class_weight
weight_for_0 = (1 / number_neg)*(total)/2.0
weight_for_1 = (1 / number_pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
#建指標
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
# DataFrame visulization
scaler = StandardScaler()
train_features = scaler.fit_transform(train_x)
val_features = scaler.transform(valid_x)
# test_features = scaler.transform(test_x)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
# test_features = np.clip(test_features, -5, 5)


# pos_df = pd.DataFrame(train_features[ bool_train_labels], columns=train_df.columns)
# neg_df = pd.DataFrame(train_features[~bool_train_labels], columns=train_df.columns)
pos_df = Y.PerStatus.value_counts()[1]
neg_df = Y.PerStatus.value_counts()[0]

sns.jointplot(x=pos_df['V5'], y=pos_df['V6'],
              kind='hex', xlim=(-5,5), ylim=(-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(x=neg_df['V5'], y=neg_df['V6'],
              kind='hex', xlim=(-5,5), ylim=(-5,5))
_ = plt.suptitle("Negative distribution")

In [ ]:
# nb_train_samples = 5217
# nb_validation_samples = 17
epochs = 1
batch_size = 16

In [ ]:
#建提早結束指令
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='precision', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
# #建model
# def model(metrics = METRICS, output_bias=None):

#     model = Sequential()
#     model.add(Dense(128, input_dim=INPUT_DIM, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(BatchNormalization())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(BatchNormalization())
#     model.add(Dense(16, activation='relu'))
#     model.add(Dropout(0.1))
#     model.add(Flatten())
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), 
#                   loss='binary_crossentropy', 
#                   metrics=metrics)
#     model.summary()
    
#     return model
#     #loss='binary_crossentropy'
#     #loss=custom_loss(recall_weight=0.9, spec_weight=0.1)

In [ ]:
def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          256, activation='relu',
          input_dim=INPUT_DIM),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [ ]:
KerasClassifier_model = KerasClassifier(build_fn=make_model, nb_epoch=EPOCHS, batch_size=BATCH_SIZE)
KerasClassifier_model._estimator_type = "classifier"

weighted_model = KerasClassifier_model
weighted_history = weighted_model.fit(
    train_x,
    train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(valid_x, valid_y),
    class_weight=class_weight)

pred_train_x= weighted_model.predict(train_x)
pred_valid_x= weighted_model.predict(valid_x)
pred_test= weighted_model.predict(test_ds_valid)

scores_train = weighted_model.score(train_x, train_y, verbose=0)
scores_valid = weighted_model.score(valid_x, valid_y, verbose=0)

print('scores_train: {:.2f}'.format(scores_train))
print('scores_valid: {:.2f}'.format(scores_valid))

In [ ]:
pred_test_pd = pd.DataFrame(pred_test)

In [ ]:
pred_train_x = pd.DataFrame(pred_train_x)
# train_y.value_counts()
pred_test_pd.value_counts()
# pred_test_pd.value_counts()

In [ ]:
# submission = pd.read_csv('/content/gdrive/MyDrive/data/submission.csv')

In [ ]:
score = F_beta_score(valid_y,pred_valid_x)
print(score)

In [ ]:
score = F_beta_score(train_y,pred_train_x)
print(score)

In [ ]:
pred_test_pd.to_csv('/content/gdrive/MyDrive/data/submission_3.csv')  

In [ ]:
# #2-class Precision-Recall curve:train
# disp = plot_precision_recall_curve(weighted_model, train_x, train_y)
# disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
make_model().summary()

In [ ]:
pred_test_pd